# Workshop Content

This notebook contains the full workflow for producing a geodemographic classification from scratch in python using k-means clustering. 

* **Data Access and Processing:**
    * Access UK census data and process using Pandas.
    * Select a specific region of interest (e.g., Liverpool City Region, Greater Manchester, Greater London).

* **Census Data Analysis and Variable selection:**
    * Select relevant census variables for clustering.
    * Standardise variables.
    * Perform correlation & variance analysis to identify potentially redundant variables.
    * Alternative variable selection methods (e.g., PCA, Autoencoders).

* **Clustering:**
    * Determine optimal number of clusters using Clustergrams.
    * Apply K-Means clustering to classify areas based on selected variables.
    * Perform top-down hierarchical clustering to divide clusters into subgroups.
    
* **Analytical Techniques:**
    * Use UMAP (Uniform Manifold Approximation and Projection) to visualise high-dimensional embeddings in 2D.

* **Visualisation and Communication:**
    * Visualise clusters and subclusters using Kepler.gl for interactive mapping.
    * Explore cluster characteristics using summary statistics and index scores.
    * Export results to various formats (GeoPackage, Parquet) for use in GIS software.
    
* **Cluster Naming with LLMs:**
    * Use Large Language Models (LLMs) to generate descriptive names and summaries for clusters based on their characteristics.

# What are Geodemographics?

Geodemographics is a method of classifying geographic areas based on the characteristics of their populations. It involves grouping areas with similar demographic, socio-economic, and lifestyle attributes into distinct categories or clusters. These classifications help in understanding the spatial distribution of different population segments and are widely used in various fields such as marketing, urban planning, public health, and social research.

Geodemographic classifications are typically created using statistical techniques such as cluster analysis, in particular k-means clustering. 

# Setup 

There are a number of python packages that need to be installed to run this notebook. It is recommended to use a virtual environment to manage these dependencies. This can be set up using the following commands in your terminal/command prompt:

```bash
# macOS / Linux
python -m venv .venv
source .venv/bin/activate
```
or 

```powershell
# Windows (PowerShell)
python -m venv .venv
.venv/Scripts/Activate.ps1
```
Once the virtual environment is activated, install the required packages using pip:

```bash
pip install --upgrade pip
pip install pandas geopandas pyarrow scikit-learn clustergram umap-learn seaborn plotly matplotlib numpy keplergl
```
this can also be done using magic commands in a Jupyter notebook cell:


In [ ]:
# !pip install pandas geopandas scikit-learn matplotlib seaborn umap-learn kepler.gl plotly openai

In [ ]:
#| warning: false

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from clustergram import Clustergram
import matplotlib.pyplot as plt
import os
import geopandas as gpd
from keplergl import KeplerGl
import umap.umap_ as umap
import plotly.express as px
from matplotlib import colors as mcolors
import seaborn as sns
import openai
import json

#set a  random seed for reproducibility
random_seed = 507


In [ ]:
#| echo: false

# Set global font sizes (affects titles, labels, ticks, etc.)
plt.rcParams.update({
    "axes.titlesize": 7,   # subplot titles
    "axes.labelsize": 8,   # x/y labels
    "xtick.labelsize": 6,  # x tick labels
    "ytick.labelsize": 6,  # y tick labels
    "legend.fontsize": 6,  # legend
    "figure.figsize": (8, 6)  # default figure size
})



# Retrieving and Preparing Data
We will be using census data from all four UK nations, which are available openly from the respective national statistics agencies (listed below). 
For this workshop, we will utilise a subset of census variables that have been unified across the four nations. These variables were used to produce a UK-wide Output Area Classification (OAC) in 2021 [@2021oac].
We will also need boundary data for the Output Areas (OAs).
All the data required can be downloaded from the Geographic Data Service Website [LINK], place the downloaded files in the `input_data` folder.

Original Data Sources:

1. **ONS Census 2021** (England & Wales):  
   - [ONS Data Service](https://www.ons.gov.uk/)
   - Downloaded from GitHub-based CSV files  

2. **NRS Census 2022** (Scotland):  
   - [Scotland's Census](https://www.scotlandscensus.gov.uk/)
   - Downloaded from GitHub-based CSV files   

3. **NISRA Census 2021** (Northern Ireland):  
   - [NISRA Website](https://www.nisra.gov.uk/)
   - Downloaded from GitHub-based CSV files 
   
4. **ONS Geoportal** for boundaries and shapefiles, including clipped EW, Scotland, and Northern Ireland geographies.


### Output Areas
Output Areas (OAs), called data zones in northern ireland and small areas in Scotland, are the smallest geographical units available openly in the UK Census. They are designed to have similar population sizes and are used to report census data. Each OA typically contains around 100-200 households. These are the base geographical units used in this classification.

![](images/OAexample.png){fig-align="center" width="500"}

### Spatial Standardisation

The data here has been normalised by the total population of each OA to give a percentage. This is important as OAs can vary in population size, and using raw counts would bias the clustering towards more populous areas.


In [ ]:
#Load the census data
variable_df = pd.read_parquet("input_data/uk_census_data.parquet")
#set Output area lookup code as index
variable_df = variable_df.set_index('OA')
#round to 2 decimal places
variable_df = variable_df.round(2)
#look at the shape of the dataset
print(f"Input data shape: {variable_df.shape}")
#look at the first few rows of the dataset
variable_df.head()

Short descriptions of the variables used in this example are found in the file `OAC_variables.csv`. The full variable descriptions can be found in the [2021 Census User Guide](https://www.ons.gov.uk/census/censustransformationprogramme/2021census/2021censususerguide).

These variables have been selected to provide a broad overview of demographic, socio-economic, and housing characteristics. They cover aspects such as age distribution, household composition, housing type, housing tenure, employment status and education levels.

In [ ]:
#load the lookup file which contains variable descriptions.
var_lookup = pd.read_csv("input_data/Variables_OAC.csv")[["No.","Variable_Name","Domain"]]
var_lookup

## Examine the data

We can plot the distribution of all the variables to get a sense of their distributions. Many of the variables are highly skewed, which is common for census data.

In [ ]:
# Use pandas histogram plotting function with seaborn aesthetics
sns.set_style("whitegrid")
nrows = int(np.ceil(len(variable_df.columns) / 3))

variable_df_withnames = variable_df.copy()
variable_df_withnames.columns = var_lookup['Variable_Name'].values[:58]
variable_df_withnames.hist(bins=30, figsize=(8, nrows*1.5), edgecolor='black', layout=(nrows, 3))
plt.tight_layout()
plt.show()

## Spatial Data

We will also need the Output Area boundaries to map the results. These can be downloaded from the ONS Geoportal. The file used here is a GeoPackage containing the 2021 Output Area boundaries for the whole of the UK, clipped to the extent of England and Wales, Scotland, and Northern Ireland.

In [ ]:
#---------
# Import spatial data
#---------

# Output Areas (Dec 2021) Boundaries Full Clipped EW (BFC) - E&W
# Manually downloaded file, same as in R
OA_2021_Boundary_EW = gpd.read_file("input_data/boundaries/Output_Areas_2021_EW_BGC_V2_-5587136561181621407.gpkg")
OA_2021_Boundary_EW = OA_2021_Boundary_EW.rename(columns={"OA21CD": "OA"})[["OA", "geometry"]]

# Output Areas 2022 Scotland (GeoJSON via Google Drive link)
OA_2022_Boundary_S = gpd.read_file("input_data/boundaries/Scotland_OA_22.gpkg")
OA_2022_Boundary_S = OA_2022_Boundary_S.rename(columns={"code": "OA"})[["OA", "geometry"]]

OA_2021_Boundary_NI = gpd.read_file("input_data/boundaries/DZ2021.geojson")
OA_2021_Boundary_NI = OA_2021_Boundary_NI.rename(columns={"DZ2021_cd": "OA"})[["OA", "geometry"]]

#---------
# Combine OA and calculate area
#---------

# Reproject to British National Grid (EPSG:27700)
OA_2021_Boundary_EW = OA_2021_Boundary_EW.to_crs(27700)
OA_2022_Boundary_S = OA_2022_Boundary_S.to_crs(27700)
OA_2021_Boundary_NI = OA_2021_Boundary_NI.to_crs(27700)


# Merge
OA_Boundaries = gpd.GeoDataFrame(
    pd.concat([OA_2021_Boundary_EW, OA_2022_Boundary_S, OA_2021_Boundary_NI], ignore_index=True),
    crs="EPSG:27700"
)

# set index to OA code
OA_Boundaries = OA_Boundaries.set_index("OA")
#---------
# Save UK OA file
#---------
OA_Boundaries.to_file("input_data/boundaries/OA_2021_22_Boundaries.gpkg", driver="GPKG")

#---------
# Load Local Authority District (LAD) for region selection
#---------
LAD_Boundaries = gpd.read_file("input_data/boundaries/Local_Authority_Districts_December_2022_UK_BGC_V2_5759908710055972638.gpkg")

### Selecting a Region
For this workshop, we will focus on a specific region of the UK to keep the analysis manageable. 

Focusing on a specific region allows us to create a more detailed and relevant geodemographic classification for that area, capturing local nuances and characteristics that may be lost in a broader national classification. For example, a [London specific OAC](https://data.geods.ac.uk/dataset/london-oac) was developed as London has a drastically different demographic composition to the rest of the United Kingdom [@loac]. 

By default we will use the Output Areas within the Liverpool City Region covering the city of liverpool and its surrounding areas. This region is prodominently urban and has a diverse population, making it an interesting case study for geodemographic classification.
If running this notebook on your own machine, you can change the region of study from the selection below.



In [ ]:
# region definitions (LAD22CD codes)
region_lads = {
    "Greater Manchester": [
        "E08000001","E08000002","E08000003","E08000004","E08000005",
        "E08000006","E08000007","E08000008","E08000009","E08000010"
    ],
    "Liverpool City Region": [
        "E06000006","E08000011","E08000012","E08000013","E08000014","E08000015"
    ],
    "Greater London": [
        "E09000001","E09000002","E09000003","E09000004","E09000005",
        "E09000006","E09000007","E09000008","E09000009","E09000010",
        "E09000011","E09000012","E09000013","E09000014","E09000015",
        "E09000016","E09000017","E09000018","E09000019","E09000020",
        "E09000021","E09000022","E09000023","E09000024","E09000025",
        "E09000026","E09000027","E09000028","E09000029","E09000030",
        "E09000031","E09000032","E09000033"
    ],
    "Scotland": [
        "S12000005","S12000006","S12000008","S12000010","S12000011",
        "S12000013","S12000014","S12000017","S12000018","S12000019",
        "S12000020","S12000021","S12000023","S12000026","S12000027",
        "S12000028","S12000029","S12000030","S12000033","S12000034",
        "S12000035","S12000036","S12000038","S12000039","S12000040",
        "S12000041","S12000042","S12000045","S12000047","S12000048",
        "S12000049","S12000050"
    ],
    "Northern Ireland": [
        "N09000001","N09000002","N09000003","N09000004","N09000005",
        "N09000006","N09000007","N09000008","N09000009","N09000010",
        "N09000011"
    ],
    "Wales": [
        "W06000001","W06000002","W06000003","W06000004","W06000005",
        "W06000006","W06000007","W06000008","W06000009","W06000010",
        "W06000011","W06000012","W06000013","W06000014","W06000015",
        "W06000016","W06000017","W06000018","W06000019","W06000020",
        "W06000021","W06000022","W06000023","W06000024"
    ],
}


Selecting the region of interest:

In [ ]:
#Choose region from the list above by uncommenting the relevant line
lad_codes = region_lads["Liverpool City Region"]
# #or eg:
# lad_codes = region_lads["Northern Ireland"]

# Spatial join to filter only intersecting OAs
oas_region = gpd.sjoin(OA_Boundaries, LAD_Boundaries[LAD_Boundaries["LAD22CD"].isin(lad_codes)], predicate="intersects")

# --- Merge OA polygons with your variable data keeping only those with matching OAs in the region
oas_region_vars = oas_region.join(variable_df, how="inner")

# #keeo only OAs in our region
variable_df_region=variable_df.loc[variable_df.index.isin(oas_region.index)]

### Map the area

In [ ]:
#| output: false

from shapely.ops import unary_union

region_geom = unary_union(
    LAD_Boundaries.loc[LAD_Boundaries["LAD22CD"].isin(lad_codes), "geometry"]
)
region_centroid = region_geom.centroid
centroid_ll = gpd.GeoSeries([region_centroid], crs=LAD_Boundaries.crs).to_crs(epsg=4326).iloc[0]
centroid_lat, centroid_lon = centroid_ll.y, centroid_ll.x

# --- Quick Kepler map ---
area_map_ = KeplerGl(
    height=600,
    config={
        "version": "v1",
        "config": {
            "mapState": {
               'latitude': centroid_lat,
                'longitude': centroid_lon,
                "zoom": 9,
                "pitch": 0,
                "bearing": 0
            }
        }
    }
)
# Add your layer
area_map_.add_data(data=oas_region_vars)
# If you want to export to a standalone HTML:
area_map_.save_to_html(file_name="outputs/maps/region_oas_map.html")
# Show inside Jupyter
area_map_

<iframe src="outputs/maps/region_oas_map.html" width="100%" height="650" style="border:none;"></iframe>

# Transform and Standardize Variables

Before applying clustering algorithms, it is important to transform and standardize the data to ensure that all variables contribute equally to the analysis.

The function below applies two transformations to a dataframe (applied column-wise):

1. **Inverse Hyperbolic Sine (IHS) transform**  
2. **Min–Max scaling to [0, 1]**

### Mathematical definitions

**Inverse hyperbolic sine (IHS, a.k.a. arcsinh):**  
- Similar to a log transform but works with zero and negative values.  
- Helps stabilize variance and make skewed distributions more normal-like.  

$$
\mathrm{arcsinh}(x) = \ln\!\big(x + \sqrt{x^{2}+1}\big)
$$

**Properties:**
- For large \(|x|\):  
  $$
  \mathrm{arcsinh}(x) \approx \ln(2|x|)
  $$
  (behaves like log).  

- Near \(0\):  
  $$
  \mathrm{arcsinh}(x) \approx x
  $$

 

**Min–Max scaling (applied per column after IHS):**

$$
x' = \frac{x - \min(x)}{\max(x) - \min(x)}
$$

- Rescales all values into the range \([0, 1]\).  
- Useful for comparing variables with different units/scales.


In [ ]:
def transform_and_standardize_data(df):
    """Apply inverse hyperbolic sine transform, to account for non-normality
    and then range standardize using min-max scaling to the dataframe."""
    df = np.arcsinh(df)
    denom = df.max() - df.min()
    df = (df - df.min()) / denom.replace(0, 1)  # prevent divide-by-zero
    return df

# Transform the input data
transformed_variable_df = transform_and_standardize_data(variable_df_region)


# Variable Selection
In geodemographics, **variable selection** is crucial to turn large datasets (like the Census) with 100s of variables into meaningful clusters.  
The nature of clustering means the high-dimensional space can be sparse and noisy, so reducing the number of variables helps improve cluster quality and interpretability.

Key points:  
- **Intention** – variables depend on the purpose of the geodem (e.g. retail vs. health).  
- **Correlation** – drop highly correlated variables to avoid redundancy.  
- **Variance** – keep variables that vary across places (so they can distinguish areas).  
- **Expert choice** – ensure selected variables are socially meaningful.  

Here we are using a **broad, pre-selected dataset** which was used for the UK OAC 2021 classification. So we expect the variables to be broadly suitable for our region of interest.

New methods include:  
- **Automated variable selection** [@pcavarselect] – uses statistical procedure to determine a subset of variables which produce the best clustering results.  
- **Autoencoders** (see Alex’s talk tomorrow!) – Use neural networks to compress all variables into a smaller set of features, while preserving the most important patterns.


In [ ]:
#| out.width: "100%"

def check_corrs(df, corr_threshold=0.9):
    # Compute correlation matrix
    corr_matrix = df.corr()

    # --- Interactive heatmap with Plotly ---
    fig = px.imshow(
        corr_matrix.values,
        color_continuous_scale="RdBu_r",
        zmin=-1, zmax=1,
        title="Correlation Heatmap",
        x=corr_matrix.columns,
        y=corr_matrix.columns,
    )
    # Add hover names for tooltips
    fig.update_traces(
        hovertemplate="<b>%{x}</b> vs <b>%{y}</b><br>Correlation: %{z:.3f}<extra></extra>",
    )
    # Hide x/y tick labels but keep tooltips
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)

    # fig.update_layout(width=800, height=800)
    fig.show()

    # --- Find highly correlated pairs (print each pair twice) ---
    cols = corr_matrix.columns
    # consider only upper triangle to get unique unordered pairs (i < j)
    upper_mask = np.triu(np.ones(corr_matrix.shape, dtype=bool), k=1)
    # build mask for correlations above threshold (and exclude perfect 1.0)
    corr_vals = corr_matrix.values
    mask = (corr_vals > corr_threshold) & (corr_vals < 1) & upper_mask

    for i, j in zip(*np.where(mask)):
        col_i, col_j = cols[i], cols[j]
        val = corr_matrix.iat[i, j]
        # print the pair twice (both directions)
        print(f"High correlation between {col_i} and {col_j}: {val:.3f}")
        print(f"High correlation between {col_j} and {col_i}: {val:.3f}")


transformed_variable_df_withnames = transformed_variable_df.copy()
transformed_variable_df_withnames.columns = [var_lookup.set_index("No.")["Variable_Name"].to_dict().get(col, col) for col in transformed_variable_df.columns]

# Check for highly correlated variables and remove them, set the thereshold as desired.
check_corrs(
    transformed_variable_df_withnames, corr_threshold=0.90)

#varience check
# Calculate variance for each column
variances = transformed_variable_df_withnames.var()
#plot the variances
plt.figure(figsize=(8, 5))
variances.sort_values().plot(kind='bar')
plt.ylabel('Variance')
plt.title('Variance of Each Variable')
plt.show()

## Removing Variables
If we want to remove any variables we can do so here. 
This could be based on the analysis above or to tailor the classification to a specific purpose.

In [ ]:
# drop_vars = ['v02', 'v04']  # Replace with actual variable names to drop
drop_vars = [] # Example: No variables to drop
# for northern ireland #bangladeshi ethnicity now removed as no variation as no bangladeshi in northern ireland
# drop_vars = ['v12'] 

cleaned_variable_df = transformed_variable_df.drop(columns=drop_vars)

# K-Means Clustering

K-means clustering is the most commonly used clustering algorithm for geodemographic classification.


K-Means is one of the most widely used clustering algorithms in unsupervised machine learning. It partitions a dataset into *k* groups (clusters), where each observation belongs to the cluster with the nearest mean (centroid). The algorithm iteratively updates cluster assignments and centroids until convergence.

**How it works (simplified):**

1. Choose the number of clusters (*k*).  
2. Initialize *k* centroids (randomly or using methods like *k-means++*).  
3. Assign each data point to the nearest centroid.  
4. Update centroids as the mean of the points in each cluster.  
5. Repeat steps 3–4 until assignments no longer change (or improvement is below a threshold).

**Strengths:**
- Simple and computationally efficient.  
- Works well when clusters are roughly spherical and similar in size.  

**Limitations:**
- Requires specifying *k* in advance.  
- Sensitive to outliers and scaling of features.  
- Assumes clusters are convex and isotropic, which may not hold in real data.  

More details and examples can be found here: [Scikit-learn: K-Means](https://scikit-learn.org/stable/modules/clustering.html#k-means)


## Choosing the number of clusters (k) - Clustergrams

When using k-means clustering, one of the key decisions is selecting the optimal number of clusters (k). This choice can significantly impact the quality and interpretability of the resulting geodemographic classification.
Key considerations when choosing k:
- Each cluster be as homogeneous as possible.
- Each cluster should be as distinct from the others as possible.
- The clusters should be as evenly sized as possible.

Clustergrams are visualisation technique that shows how cluster assignments change as you increase the number of clusters (k). This helps you to understand the structure in very high-dimensional space in the following ways:

- **Cluster separation**: Helps you to determine the right number of clusters by visualising how cleanly clusters separate  
- **Cluster stability**: Shows which clusters persist across different k values (stable long lines) vs. those which are artifacts of over-clustering (short, erratic lines)  
- **Split patterns**: Reveals the natural hierarchy in the data by showing how clusters subdivide  


Further guidance on interpreting clustergrams and choosing the number of clusters can be found here:  [Clustergram](https://clustergram.readthedocs.io/en/stable/notebooks/introduction.html)

In [ ]:
def create_clustergram(df, n_init, save_loc, random_seed=random_seed):
    """
    Create and save a clustergram for evaluating k-means clustering solutions.

    The clustergram visualizes clustering stability and helps identify the optimal 
    number of clusters by performed the k-means algorithm for a range of cluster
    numbers.
    Since k-means is sensitive to initialization, `n_init` determines the number of 
    times the algorithm runs with different centroid seeds. The final result is the 
    best outcome based on inertia/WCSS (within-cluster sum of squares.

    Parameters:
    df (pd.DataFrame or np.ndarray): The input data for clustering.
    n_init (int): Number of k-means runs with different initial centroid seeds. 
                  Higher values (e.g., ~1000) improve solution stability but increase runtime.
    save_loc (str): File path to save the clustergram plot.
    random_seed (int, optional): Random seed for reproducibility.
    """
    cgram = Clustergram(range(1, 10), n_init=n_init, random_state=random_seed,verbose=False)  # Initialize clustergram model
    cgram.fit(df)  # Fit model to data
    cgram.plot()  # Generate plot
    plt.savefig(save_loc)  # Save figure
    plt.show()  # Display plot

# Example usage
n_init = 100  # Use a low value for quick testing, increase (~100) for final results
create_clustergram(cleaned_variable_df, n_init, save_loc="outputs/plots/supergroup_clustergram.png")


Choose the number of clusters (k) based on the clustergram above. 

In [ ]:
# Define the number of clusters (K). Choose K based on the clustergram plot.
num_clusters = 5

In [ ]:
def run_kmeans(input_df, num_clusters, n_init, output_filepath, random_seed=None):
    """
    Run K-means clustering on the input dataset and save the cluster assignments.

    This function applies K-means clustering to the provided dataset, assigns cluster 
    labels to each row, and saves the cluster assignments as a lookup table.

    Parameters:
    input_df (pd.DataFrame): The input dataset to be clustered.
    num_clusters (int): The number of clusters (K) to create.
    n_init (int): Number of times the K-means algorithm runs with different initial 
                  centroid seeds. The best result based on inertia/WCSS is chosen. 
                  A higher value (e.g., ~1000) is recommended for final results, 
                  but a lower value can be used for testing.
    output_filepath (str): Path to save the resulting cluster assignments.
    random_seed (int, optional): Random seed for reproducibility.

    Returns:
    pd.DataFrame: The input DataFrame with an added 'cluster' column containing 
                  the assigned cluster for each row.
    """
    df = input_df.copy()
    # Initialize the K-means model
    kmeans_model = KMeans(n_clusters=num_clusters,init="random", random_state=random_seed, n_init=n_init)
    
    # Fit the model and assign clusters
    df['cluster'] = kmeans_model.fit_predict(df)

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)

    # Save the cluster assignments to a CSV file
    df[['cluster']].to_csv(output_filepath)
    return df


n_init = 100 # Use a low value for quick testing, increase (~100) for final results
output_filepath = "outputs/supergroups_clusteroutput.csv"
print(f"Output file path: {output_filepath}")

# Run K-means clustering
supergrouped_variable_df = run_kmeans(cleaned_variable_df, num_clusters, n_init=n_init, output_filepath = output_filepath, random_seed=random_seed)

# Map numeric labels to letters
label_map = {i: chr(65 + i) for i in range(num_clusters)}  # 0->A, 1->B, etc.
supergrouped_variable_df['cluster'] = supergrouped_variable_df['cluster'].map(label_map)

#supergrouped_variable_df contains the cluster assignments for each row in the input data, and the input data itself.
supergrouped_variable_df["cluster"].head()


Visualise the clusters on a map.

In [ ]:
#| output: false
#| echo: false


# Custom cluster colours for up to 10 clusters
colours = {
    "A": '#8dd3c7',
    "B": '#ffffb3',
    "C": '#bebada',
    "D": '#fb8072',
    "E": '#fdb462',
    "F": "#235477",
    "G": '#fccde5',
    "H": '#d9d9d9',
    "I": '#bc80bd',
    "J": '#ccebc5'
}

sorted_clusters = sorted(colours.keys())
colour_list = [colours[k] for k in sorted_clusters]

# #append geometry column from oas_region to supergrouped_variable_df to make a geodataframe
gdf = oas_region.merge(supergrouped_variable_df, left_index=True, right_index=True, how='left')

# Config for map
config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [],
            "layers": [
                {
                    "id": "clusters_layer",
                    "type": "geojson",   # polygon layer
                    "config": {
                        "dataId": "clusters",
                        "label": "Clusters",
                        'columns': {'geojson': 'geometry'},
                        "color": [130, 154, 227],
                        "highlightColor": [252, 242, 26, 255],
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "thickness": 0,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Custom",
                                "type": "qualitative",
                                "category": "Custom",
                                "colors": colour_list
                            },
                            "filled": True
                        },
                        "hidden": False,
                        "textLabel": []
                    },
                    "visualChannels": {
                        "colorField": {"name": "cluster", "type": "string"},
                        "colorScale": "ordinal",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear"
                    }
                }
            ],
            "effects": [],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "clusters": [
                            {"name": "cluster", "format": None}
                        ]
                    },
                    "enabled": True
                },
                "legend": {
                    "enabled": True,
                    "active": True
                }
            },

            "layerBlending": "normal"
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
               'latitude': centroid_lat,
            'longitude': centroid_lon,
            "pitch": 0,
            "zoom": 9,
            "isSplit": False
        },
        "mapStyle": {
            "styleType": "dark-matter",
            "topLayerGroups": {
                "water": True,
                "building": True
            },
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True,
                "3d building": False
            },
            "mapStyles": {
                "dark-matter": {
                    "id": "dark-matter",
                    "label": "Carto Dark Matter",
                    "url": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json"
                }
            }
        }
    }
}

# Show in Kepler
map_with_basemap = KeplerGl(data={"clusters":gdf}, config=config, height=600)
map_with_basemap.save_to_html(file_name="outputs/maps/cluster_map.html")
map_with_basemap

<iframe src="outputs/maps/cluster_map.html" width="100%" height="650" style="border:none;"></iframe>

## UMAP Visualization

We can use UMAP (Uniform Manifold Approximation and Projection) to visualise the high-dimensional census data in 2D. UMAP is a dimensionality reduction technique that preserves both local and global structure in the data, making it well-suited for visualising complex datasets like census data.

In [ ]:
# Features = all columns except the cluster label
features = [c for c in supergrouped_variable_df.columns if c != 'cluster']
# Extract features and labels  (transformed)
X = supergrouped_variable_df[features].values
labels = supergrouped_variable_df['cluster'].values

# Fit UMAP
# Apply UMAP to reduce 64 dimensions to 2D
reducer = umap.UMAP(
    n_neighbors=30,        # Numbers of neighbours
    min_dist=0.0,          # Allow points to be closer together
    n_components=2,        # Reduce to 2D for visualsation
    random_state=random_seed,       # For reproducible results
    metric='cosine',       # Cosine similarity works well for embeddings
    init='random',         # Use random initialisation
    n_epochs=500,          # More epochs for better convergence
    spread=1.0,            # Controls how tightly points are packed
    verbose=False          # Show progress
)

embedding = reducer.fit_transform(X)

umap_results = pd.DataFrame({
    'UMAP1': embedding[:, 0],
    'UMAP2': embedding[:, 1],
    'Cluster': labels
})

# Save the UMAP results
umap_results.to_parquet('outputs/umap_results.parquet', index=False)

In [ ]:
#| out.width: "100%"


# Define colours for each cluster - same as earlier map
colours = {
    "A": '#8dd3c7',
    "B": '#ffffb3',
    "C": '#bebada',
    "D": '#fb8072',
    "E": '#fdb462',
    "F": "#235477",
    "G": '#fccde5',
    "H": '#d9d9d9',
    "I": '#bc80bd',
    "J": '#ccebc5'
}

# Create interactive UMAP scatter plot
fig_interactive = px.scatter(
    umap_results,
    x='UMAP1',
    y='UMAP2',
    color='Cluster',
    category_orders={"Cluster": sorted(umap_results["Cluster"].unique())},  #
    color_discrete_map=colours,  
)

# Style tweaks
fig_interactive.update_traces(marker=dict(size=3, opacity=0.7))
fig_interactive.update_layout(
    title="UMAP Projection of Clusters",
    xaxis_title="UMAP 1",
    yaxis_title="UMAP 2",
    legend_title="Cluster"
)


#save to html
fig_interactive.write_html("outputs/umap_interactive.html")
fig_interactive.update_layout(width=800, height=600)
fig_interactive.show(config={"responsive": False})

The UMAP projection indicates further structure within clusters
More on that in a bit.. for now lets dig into the clusters that we've got.

# Cluster Profiling, Naming, and Describing using Language Models

We can explore the characteristics of each cluster using summary statistics and index scores. This helps us understand each cluster.

## Cluster Statistics

In [ ]:
#Lot at the characteristics of each cluster

# Read in the data
pop_size = pd.read_csv("input_data/oa_pop_data.csv")
pop_size = pop_size.set_index('OA')
#rename column to "population"
pop_size = pop_size.rename(columns={'uk001001': 'population'})
pop_size = pop_size['population']

#basic statistics of each cluster, number (perc of OAs) in each cluster and population

#number of OAs in each cluster
cluster_counts = supergrouped_variable_df['cluster'].value_counts().sort_index()
#percentage of OAs in each cluster
cluster_perc = (cluster_counts / cluster_counts.sum() * 100).round(2)

#join pop_size to supergrouped_variable_df on index
supergrouped_variable_df_withpop = supergrouped_variable_df.join(pop_size, how='left')

#pop in each cluster
cluster_pop = supergrouped_variable_df_withpop.groupby('cluster')['population'].sum()
#percentage of pop in each cluster
cluster_pop_perc = (cluster_pop / cluster_pop.sum() * 100).round(2)

#combine into a dataframe
cluster_summary = pd.DataFrame({
    'Number of OAs': cluster_counts,
    'Percentage of OAs': cluster_perc,
    'Population': cluster_pop,
    'Percentage of Population': cluster_pop_perc
})
cluster_summary

## Cluster Profiling

Index scores are a way to summarise how a particular variable behaves within a cluster compared to the overall average. They help identify which characteristics are over- or under-represented in each cluster.

Index scores are calculated as follows:

$$
\text{Index Score} = \left( \frac{\text{Mean of Variable in Cluster}}{\text{Overall Mean of Variable}} \right) \times 100
$$

Where:

- **Mean of Variable in Cluster**: the average value of the variable for all areas within the specific cluster.  
- **Overall Mean of Variable**: the average value of the variable across all areas in the dataset.  

Here we will look only at variables used in the clustering. It can also be useful to look at variables not used in the clustering or from other data sources to help understand the clusters.


In [ ]:
# map encoding -> human name
encoding_to_name = dict(zip(var_lookup["No."], var_lookup["Variable_Name"]))

features = [c for c in supergrouped_variable_df.columns if c != 'cluster']
#dont average the cluster column
cluster_means = supergrouped_variable_df.groupby('cluster').mean()
global_means = supergrouped_variable_df[features].mean()
global_stds = supergrouped_variable_df[features].std()

# --- Calculate percentage difference ---
pct_diff = (cluster_means / global_means) * 100
#drop columns with nan
pct_diff = pct_diff.dropna(axis=1, how='any')
pct_display_df = pct_diff.T  # index: encodings


# --- Calculate percentage difference ---
pct_diff = (cluster_means / global_means) * 100
#drop columns with nan
pct_diff = pct_diff.dropna(axis=1, how='any')
pct_display_df = pct_diff.T  # index: encodings

# build customdata for hover (human names repeated across clusters)
human_names = pct_display_df.index.map(lambda e: encoding_to_name.get(e, e)).values
customdata_pct = np.tile(human_names.reshape(-1, 1), (1, pct_display_df.shape[1]))

# get symmetric range around 100
max_abs = np.nanmax(np.abs(pct_display_df.values - 100))

# --- Heatmap (percentage difference) ---d
fig_pct = px.imshow(
    pct_display_df,
    color_continuous_scale="RdYlGn",
    origin="lower",
    aspect="auto",
    labels=dict(x="Cluster", y="Feature (encoding)", color="% of global mean"),
    zmin=0,
    zmax=200
)

# attach customdata and set hover
fig_pct.data[0].customdata = customdata_pct
fig_pct.update_traces(
    hovertemplate="Cluster: %{x}<br>Encoding: %{y}<br>Name: %{customdata}<br>% of Global Mean: %{z:.1f}%<extra></extra>",
    zmid=100  # centre colours on 100%
)

fig_pct.update_layout(
    title="Cluster Profiles (% of Global Mean)",
    xaxis_title="Cluster",
    yaxis_title="Feature (encoding)",
    height=800
)

fig_pct.show()


## LLM Cluster Naming (and description)

To create a useful geodemographic classification, we need to assign meaningful names and descriptions to each cluster. This helps in interpreting the clusters and communicating their characteristics effectively. 
Traditionally, this is done manually by examining the statistical profiles of each cluster (often with input (as we produced) sometimes with external ) and using domain knowledge to assign names. Either done by a single expert, panel or utilising crowd sourcing approaches. Either way it is a time consuming process. However, we can leverage Large Language Models (LLMs) to assist in this process. We have demonstrated that LLMs can be used to generate initial name and description suggestions based on the statistical profiles of each cluster [@llmpaper].
We can use LLMs to generate initial name and description suggestions based on the statistical profiles of each cluster.
Again here we are using only the variables used in the clustering. It can also be useful to include other variables or external data to provide more context for the LLM.

### Using the OpenAI API
Below I use the OpenAI API, if you have an API key insert it in an .env file as;
OPENAI_API_KEY="sk....sA"

If you do have an API key skip the next cell and go to the cell where we will use a browser prompt to get the cluster names and descriptions.

In [ ]:
#| eval: false
#| echo: true
#| output: true

#get your OpenAI API key from environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

client = openai.OpenAI(api_key=openai_api_key)

# -------------------------
# JSON Schema for output
# -------------------------
cluster_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "description": {"type": "string"},
    },
    "required": ["name", "description"],
    "additionalProperties": False,
}


system_prompt = """You are a geodemographic analyst. 
Your task is to produce commercial-style geodemographic cluster pen portraits 
and cluster names."""

user_prompt = """
A geodemographics company is trying to explain the characteristics of several 
neighborhoods to a new customer. They present data comparing each 
neighborhood to the region average. A score of 100 means the neighborhood 
is equivalent to the regional average, 150 means one and a half times, 200 means twice, 50 means half, 
and 300 means three times the regional average. 

The description of each neighborhood should focus on characteristics with scores above 120 or below 80. 
Write in the third person, no more than 500 words. 
Do not mention the specific scores. Instead, describe patterns relative to the regional average (above/below). 

In the style of a commercial geodemographic classification, create a cluster name that summarises the pen portrait. 
The name should capture as many different characteristics as possible and be no more than 3 words.
"""

# -------------------------
# Loop through clusters
# -------------------------
cluster_summaries = {}
for cluster in pct_diff.index:
    cluster_pct = pct_diff.loc[cluster]

    cluster_data = {
        "cluster": cluster,
        "data": {
            encoding_to_name.get(feature, feature): round(value, 1)
            for feature, value in cluster_pct.items()
        },
    }

    response = client.responses.create(
        model="gpt-5-mini",
        input=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_prompt.strip()},
            {"role": "user", "content": json.dumps(cluster_data)},
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "cluster_summary",
                "schema": cluster_schema,
                "strict": True,
            }
        },
    )

    cluster_summary = json.loads(response.output_text)

    # store under your cluster ID
    cluster_summaries[cluster] = cluster_summary

cluster_summaries


### Prompt for browser based LLM
The follow code cell generates a prompt for to be used in an LLM of your choice.
Try it in your browser based LLM of choice (e.g. chatGPT, Claude, Gemini, etc)

The prompt should insure that the LLM produses the output in the correct format but this cannot beguaranteed.

In [ ]:
#print the prompt to copy and paste into a LLM to generate cluster descriptions

prompt_intial = """
A geodemographics company is trying to explain the characteristics of several neighborhoods to a new customer. 
They present data comparing each neighborhood to the region average. 
A score of 100 means the neighborhood is equivalent to the national average, 
a score of 150 means the neighborhood is one and a half times the national average, 
a score of 200 means the neighborhood is twice the national average, 
a score of 50 means the neighborhood is half of the region average, 
a score of 300 means the neighborhood is three times the region average. 

Each neighborhood has the following characteristics, described in #DATA# below. 
Data are presented for each characteristic followed by a colon, and then a score. 
The description of each neighborhood should focus on characteristics that have scores which are greater than 120 or less than 80.
Write a separate description for each cluster (Cluster A, Cluster B, Cluster C, Cluster D, etc. 
Each description should be written in the third person, in no more than 500 words. Do not mention the specific scores from the #DATA#. 
Instead, use descriptive words to illustrate rates that are above or below the regional average.
Make comparisons to the regional average, do not talk in absolute terms.
"""

prompt_data =""
# print the index scores for each cluster in this format:

for cluster in pct_diff.index:
    prompt_data += f"\n#DATA# cluster_key: {cluster}\n"
    cluster_pct = pct_diff.loc[cluster]
    for feature, value in cluster_pct.items():
        feature_name = encoding_to_name.get(feature, feature)
        prompt_data += f"{feature_name}: {value:.1f}\n"

prompt_struc = """
In the style of a commercial geodemographic classification; create a cluster name 
that would summarise the created geodemographic pen portraits. The names should capture as many 
different characteristics contained within the description as possible. 
The cluster name should be no more than 3 words.
Return your response in JSON format with the structure: {"cluster_key_1": {"name": "", "description": ""},"cluster_key_2": {"name": "", "description": ""},...}"""

full_prompt = prompt_intial + prompt_data + prompt_struc
print(full_prompt)

### Copy result in here;

In [ ]:
cluster_summaries = {
  "A": {
    "name": "Elderly Communal Flats",
    "description": "Cluster A neighborhoods feature a notably higher proportion of very elderly residents compared to the regional average. They also have significantly more individuals living in communal establishments than typical in the region. Housing in these areas shows a markedly elevated presence of flats, maisonettes, or apartments relative to the regional norm. On the demographic side, these neighborhoods exhibit substantially lower representation of Black ethnic groups than the regional average. Additionally, the rate of residents who cannot speak English well or at all is considerably below the regional average, indicating better overall English proficiency among the population."
  },
  "B": {
    "name": "Affluent Rural Suburban",
    "description": "Cluster B neighborhoods are characterized by a significantly higher prevalence of detached houses compared to the regional average, suggesting a more spacious and possibly rural or suburban housing landscape. These areas have notably fewer residents born in EU countries, non-EU European countries, or Africa than the regional norm, pointing to lower levels of international migration. Ethnically, they show substantially reduced proportions of Bangladeshi, Pakistani, other Asian, Black, and mixed or multiple ethnic groups relative to the regional average. The rate of individuals who cannot speak English well or at all is markedly lower than typical, implying stronger English language skills across the population. Religious diversity is less pronounced, with other religions represented at a considerably lower rate than the region. Household structures lean towards fewer one-person households and more families without children, but the standout is a dramatically reduced presence of communal living establishments. Housing types reflect fewer terraced houses, flats, maisonettes, or apartments, as well as substantially lower social and private renting compared to the regional average. Health indicators, such as standardized illness ratios, are better than the regional norm, indicating lower illness rates. Unemployment is also notably below the regional average, contributing to a more stable economic environment."
  },
  "C": {
    "name": "Diverse Urban Renters",
    "description": "Cluster C neighborhoods stand out with significantly higher proportions of residents born in EU countries, non-EU European countries, and Africa compared to the regional average, reflecting substantial international diversity. Ethnically, these areas have markedly elevated representations of Bangladeshi, Chinese, Indian, Pakistani, other Asian, Black, and mixed or multiple ethnic groups relative to the regional norm, while White ethnic groups are somewhat less prominent. The rate of residents who cannot speak English well or at all is considerably above the regional average, suggesting challenges in language proficiency. Other religions are more common than typical in the region. Housing features a notably higher presence of terraced houses, including end-terraces, and flats, maisonettes, or apartments compared to the average. Tenure shows elevated levels of social renting and private renting beyond the regional norm, with ownership or shared ownership being less prevalent. Vehicle ownership is lower, with fewer households having two or more cars or vans than the regional average. Unemployment rates are substantially higher than the region, indicating potential economic pressures. Additionally, the proportion of very elderly residents is below the regional average, contributing to a younger demographic tilt."
  },
  "D": {
    "name": "Working-Class Terraced Renters",
    "description": "Cluster D neighborhoods exhibit significantly lower proportions of residents born in non-EU European countries and Africa compared to the regional average, indicating reduced international origins from these areas. Ethnically, representations of Bangladeshi, Chinese, Indian, Pakistani, and other Asian groups are markedly below the regional norm. Living arrangements show fewer individuals in communal establishments than typical. Housing types feature a notably higher presence of terraced houses, including end-terraces, relative to the average, while detached houses or bungalows are less common. Social renting is elevated beyond the regional norm. Health indicators like standardized illness ratios are somewhat higher than average, suggesting more health challenges, though not drastically so. Unemployment is also above the regional average, pointing to economic vulnerabilities."
  },
  "E": {
    "name": "Transient Cosmopolitan Students",
    "description": "Cluster E neighborhoods are distinguished by substantially higher proportions of residents born in EU countries, non-EU European countries, and Africa compared to the regional average, highlighting intense global diversity. Ethnically, these areas feature markedly elevated representations of Bangladeshi, Chinese, Indian, Pakistani, other Asian, Black, and mixed or multiple ethnic groups relative to the norm. The rate of individuals who cannot speak English well or at all is significantly above average, indicating notable language barriers. Religiously, no religion is more prevalent, while Christianity is considerably less common, and other religions are much more represented than in the region. Marital status shows higher rates of never married individuals and lower rates of married or civil partnered residents, as well as fewer separated or divorced. Households have more one-person setups and fewer with dependent children compared to the average, with substantially reduced instances of all members sharing the same ethnic group. Communal establishments are dramatically more common, and residential stability is lower, with fewer people living at the same address as the previous year. Housing leans heavily towards flats, maisonettes, or apartments at rates far above the regional norm, while detached, semi-detached, and terraced houses are all significantly less prevalent. Ownership is below average, but private renting is notably higher. Demographically, proportions of young children, school-aged children, middle-aged adults, and elderly residents are all markedly lower than the region, creating a younger adult focus. Provision of unpaid care is less common. Vehicle ownership is reduced, with fewer households having two or more cars. Educationally, highest qualifications at level 1-2 or apprenticeship are below average, but level 4 or above are higher, alongside a substantially elevated full-time student population. Occupationally, skilled trades and process plant/machine operatives are less represented than the regional average."
  }
}
cluster_descriptions_df = pd.DataFrame.from_dict(cluster_summaries, orient='index')


#pretty print the descriptions (break lines for readability)
for cluster, row in cluster_descriptions_df.iterrows():
    print(f"Cluster {cluster} - {row['name']}:\n")
    description = row['description']
    #break into lines of max 80 characters
    import textwrap
    wrapped_description = textwrap.fill(description, width=70)
    print(wrapped_description, "\n\n")

## Save results
Lets save the results to file for use in GIS software or to format for sharing.


# Hierarchical Subclustering

We often want to perform a finer level of clustering to capture more detailed patterns in the data.
For OAC the top level "supergroup" clusters are split further into groups and subgroups by applying the above process iteratively. This process is referred to as top down clustering. This has the advantage of allowing more clusters to be created without needing to consider all clusters at once. It also allows for more interpretable clusters as the subclusters are nested within the broader supergroup clusters.

## Selecting the number of subclusters for each supergroup using Clustergrams

In [ ]:
def create_subcluster_clustergrams(output_df, num_clusters, n_init=1):
    """
    Generate and save clustergrams for each supercluster.
    This function loops through the existing clusters and creates a clustergram 
    for each
    Parameters:
    output_df (pd.DataFrame): DataFrame containing cluster assignments.
    num_clusters (int): The total number of clusters to iterate over.
    n_init (int, optional): Number of times K-means runs with different centroid seeds.
                            Defaults to 1 for quick testing.

    """
    save_dir = "outputs/plots" #directory to save the clustergrams
    os.makedirs(save_dir, exist_ok=True)  # Ensure save directory exists
    cluster_labels = np.sort(output_df["cluster"].unique())
    print(cluster_labels)
    for cluster_label in cluster_labels:
        # Select rows corresponding to the current cluster, dropping the 'cluster' column
        cluster_df = output_df.query("cluster == @cluster_label").drop(columns='cluster')

        print(f"Cluster: {cluster_label,cluster_summaries[cluster_label]["name"]}, {len(cluster_df)} geographies in cluster")

        if cluster_df.empty:
            print(f"Skipping cluster {cluster_label} as it has no data.")
            continue

        # Define save location
        save_loc = os.path.join(save_dir, f"subcluster_clustergram_cluster{cluster_label}.png")
        print(f"Saving clustergram to {save_loc}")

        # Generate clustergram
        create_clustergram(cluster_df, n_init=n_init, save_loc=save_loc)

# Example usage
create_subcluster_clustergrams(supergrouped_variable_df, num_clusters, n_init=1)



## Run the subclustering 
We can now select the number of subclusters to split each of the supergroups into using the clustergrams above.
The length of the list must match num_clusters (the number of supergroups).

In [ ]:
subcluster_nums = [2, 3, 3, 2, 2]

In [ ]:

def run_subclustering(input_df: pd.DataFrame, subcluster_nums: list, num_clusters: int, n_init: int = 1) -> pd.DataFrame:
    """
    Runs subclustering for each supergroup using KMeans and returns a modified DataFrame with subcluster labels.
    
    Parameters:
    - output_df (pd.DataFrame): The original DataFrame containing data and cluster assignments.
    - subcluster_nums (list): A list specifying the number of subclusters to split each supergroup into.
    - num_clusters (int): The total number of supergroups.
    - n_init (int, optional): The number of times KMeans will be initialized. Defaults to 1.

    Returns:
    - pd.DataFrame: A new the output dataFrame with an added 'subcluster' column.
    """

    cluster_labels = np.sort(input_df["cluster"].unique())
    print(f"Cluster labels found: {cluster_labels}")
    if len(subcluster_nums) != len(cluster_labels):
        raise ValueError(f"Length of subcluster_nums ({len(subcluster_nums)}) does not match num_clusters ({len(cluster_labels)}).")

    # Work on a copy of the DataFrame to prevent unintended modifications
    df = input_df.copy()

    for cluster, num_subclusters in zip(cluster_labels, subcluster_nums):
        print(f"Clustering supergroup {cluster,cluster_summaries[cluster]["name"]} into {num_subclusters} subclusters.")

        # Select rows corresponding to the current cluster, drop the cluster column before clustering
        cluster_df = input_df.query("cluster == @cluster").drop(columns='cluster').copy()
        # Run KMeans clustering for the selected supergroup
        subcluster_output_df = run_kmeans(
            cluster_df, 
            num_subclusters, 
            n_init=n_init, 
            output_filepath=f"outputs/subclusters/supergroup{cluster}_subclusteroutput.csv"
        )

        # Combine anems
        subcluster_output_df['subcluster'] = [str(cluster) + str(i) for i in subcluster_output_df['cluster']]

        # Update the modified DataFrame with subclustering results
        df.loc[cluster_df.index, 'subcluster'] = subcluster_output_df['subcluster']

    # Save the final output
    df.to_csv("outputs/subgroups_clusteroutput.csv")
    print("Final output saved to outputs/subgroups_clusteroutput.csv")

    return df  # Return the modified DataFrame with clusters and subclusters
subgrouped_variable_df = run_subclustering(supergrouped_variable_df, subcluster_nums, num_clusters=num_clusters, n_init=1)

## Visualise and save the results

In [ ]:
# --- Calculate percentage difference (subclusters vs cluster means) ---

# cluster means for reference
cluster_means = subgrouped_variable_df.groupby("cluster").mean(numeric_only=True)
global_means = subgrouped_variable_df[features].mean()
# subcluster means
subcluster_means = subgrouped_variable_df.groupby(["cluster", "subcluster"]).mean(numeric_only=True)

# percentage difference: subcluster relative to parent cluster
pct_diff_sub = (subcluster_means / cluster_means) * 100
pct_diff_sub = (subcluster_means/ global_means)*100
pct_display_df_sub = pct_diff_sub.T  # index = features (encodings), columns = MultiIndex (cluster, subcluster)
#replace the column MultiIndex with a single level index with "cluster-subcluster" format and swap in the cluster names from cluster_summaries
# pct_display_df_sub.columns = [f"{c[0]}-{c[1]}" for c in pct_display_df_sub.columns]
pct_display_df_sub.columns = [f"{cluster_summaries[c[0]]['name']}-{c[1]}" for c in pct_display_df_sub.columns]

# pct_display_df_sub.columns = [f"{c[0]}-{c[1]}" for c in pct_display_df_sub.columns]

# build customdata for hover (human names repeated across cluster–subcluster combos)
human_names = pct_display_df_sub.index.map(lambda e: encoding_to_name.get(e, e)).values
customdata_pct_sub = np.tile(human_names.reshape(-1, 1), (1, pct_display_df_sub.shape[1]))

# get symmetric range around 100
max_abs_sub = np.nanmax(np.abs(pct_display_df_sub.values - 100))

# --- Heatmap (percentage difference: subcluster vs cluster mean) ---
fig_pct_sub = px.imshow(
    pct_display_df_sub,
    color_continuous_scale="RdYlGn",
    origin="lower",
    aspect="auto",
    labels=dict(x="Subcluster", y="Feature (encoding)", color="% of cluster mean"),
    zmin=0,
    zmax=200
)

# attach customdata and set hover
fig_pct_sub.data[0].customdata = customdata_pct_sub
fig_pct_sub.update_traces(
    hovertemplate="Subcluster: %{x}<br>Encoding: %{y}<br>Name: %{customdata}<br>% of Cluster Mean: %{z:.1f}%<extra></extra>",
    zmid=100  # centre colours on 100%
)

fig_pct_sub.update_layout(
    title="Subcluster Profiles (% of Cluster Mean)",
    xaxis_title="Subcluster",
    yaxis_title="Feature (encoding)",
    height=800
)

# get mapping of column → cluster
col_clusters = [col.split("-")[0] for col in pct_display_df_sub.columns]

# find where cluster changes (between adjacent columns)
boundaries = [
    i + 0.5 for i in range(len(col_clusters) - 1)
    if col_clusters[i] != col_clusters[i + 1]
]

# add vertical lines at these boundaries
for b in boundaries:
    fig_pct_sub.add_vline(
        x=b, line_width=2, line_dash="dash", line_color="black"
    )

fig_pct_sub.show()


Save the results to file for use in GIS software or to format for sharing:

In [ ]:
#merge the geometry column from oas_region to subgrouped_variable_df to make a geodataframe
gdf = oas_region.merge(subgrouped_variable_df  , left_index=True, right_index=True, how='left')
#save to file
gdf.to_file("outputs/subclusters_geodataframe.gpkg", layer='subclusters', driver="GPKG")

## Map the subclusters

In [ ]:
#| output: false
#| echo: false

# Custom cluster colours for subclusters
def generate_shades(hex_color, n_shades=5, step=0.15):
    """Generate shades centred around a base color."""
    base = np.array(mcolors.to_rgb(hex_color))
    mid = n_shades // 2
    shades = []
    for i in range(n_shades):
        factor = 1 + (i - mid) * step
        shade = np.clip(base * factor, 0, 1)
        shades.append(mcolors.to_hex(shade))
    return shades

#same as before 
base_colors = {
    "A": '#8dd3c7',
    "B": '#ffffb3',
    "C": '#bebada',
    "D": '#fb8072',
    "E": '#fdb462',
    "F": "#235477",
    "G": '#fccde5',
    "H": '#d9d9d9',
    "I": '#bc80bd',
    "J": '#ccebc5'
}

# Generate subgroup colours
colors = {}
for (group, hex_color), n_sub in zip(base_colors.items(), subcluster_nums):
    shades = generate_shades(hex_color, n_shades=n_sub)
    for i, shade in enumerate(shades, start=1):
        colors[f"{group}{i}"] = shade


sorted_clusters = sorted(colors.keys())
color_list = [colors[k] for k in sorted_clusters]

# # Ensure cluster column is string for Kepler
# supergrouped_variable_df["cluster"] = supergrouped_variable_df["cluster"].astype(str)
# #append geometry column from oas_liv to supergrouped_variable_df to make a geodataframe
gdf = oas_region.merge(subgrouped_variable_df  , left_index=True, right_index=True, how='left')


# Config for polygons (fill only, no stroke)
config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [],
            "layers": [
                {
                    "id": "clusters_layer",
                    "type": "geojson",   # polygon layer
                    "config": {
                        "dataId": "clusters",
                        "label": "Clusters",
                        'columns': {'geojson': 'geometry'},
                        "color": [130, 154, 227],
                        "highlightColor": [252, 242, 26, 255],
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "thickness": 0,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Custom",
                                "type": "qualitative",
                                "category": "Custom",
                                "colors": color_list
                            },
                            "filled": True
                        },
                        "hidden": False,
                        "textLabel": []
                    },
                    "visualChannels": {
                        "colorField": {"name": "subcluster", "type": "string"},
                        "colorScale": "ordinal",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear"
                    }
                }
            ],
            "effects": [],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "clusters": [
                            {"name": "subcluster", "format": None}
                        ]
                    },
                    "enabled": True
                }
            },
            "layerBlending": "normal"
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
               'latitude': centroid_lat,
                'longitude': centroid_lon,
            "pitch": 0,
            "zoom": 9,
            "isSplit": False
        },
        "mapStyle": {
            "styleType": "dark-matter",
            "topLayerGroups": {
                "water": True,
                "building": True
            },
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True,
                "3d building": False
            },
            "mapStyles": {
                "dark-matter": {
                    "id": "dark-matter",
                    "label": "Carto Dark Matter",
                    "url": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json"
                }
            }
        }
    }
}

# Show in Kepler
map_with_basemap = KeplerGl(data={"clusters":gdf}, config=config, height=600)
map_with_basemap.save_to_html(file_name="outputs/maps/subcluster_map.html")

map_with_basemap

<iframe src="outputs/maps/subcluster_map.html" width="100%" height="650" style="border:none;"></iframe>